In [1]:
import ipynb.fs.full.training as training
import ipynb.fs.full.splitting as splitting
import ipynb.fs.full.misc as misc

In [2]:
import numpy as np

### REGRESSION ENSEMBLE

In [3]:
def regression(primary_dataset, config):
    
    # REQUESTED MODELS
    models = config['regression_ensemble']['models']
    
    # ENSEMBLE CONTAINER
    ensemble = [None] * len(models)
    
    # LOOP THROUGH MODELS
    for index, item in enumerate(models):

        # MODEL PROPS
        name, settings = misc.key_value(item)

        # FOLD CONTAINER
        folds = []

        # IF THE MODEL HAS EXTRA SETTINGS
        if settings:
            folds = splitting.timeseries(
                primary_dataset['train'],
                config['splitting']['validation_folds'],
                window=settings['morph']['window']
            )

        # OTHERWISE
        else:
            folds = splitting.timeseries(
                primary_dataset['train'],
                config['splitting']['validation_folds']
            )

        # FOLD PREDICTIONS
        temp_predictions = []
        temp_labels = []
        temp_models = []

        # LOOP THROUGH FOLDS
        for fold_index, fold in enumerate(folds):
            
            # PRINT A MESSAGE
            print('TRAINING {} FOLD #{}'.format(name.upper(), fold_index + 1))
            
            # TRAIN & PREDICT WITH THE MODEL
            model, predictions = training.start(fold, name, settings)

            # APPEND TO COLLECTIONS
            temp_predictions.append(predictions)
            temp_labels.append(fold['test']['labels'])
            temp_models.append(model)
            
        # PRINT A SPACE
        print('')

        # APPEND RESULTS TO ENSEMBLE
        ensemble[index] = {
            'name': '{}_{}'.format(name, index),
            'predictions': np.concatenate(temp_predictions),
            'labels': np.concatenate(temp_labels),
            'models': temp_models,
            'settings': settings
        }
    
    return ensemble

### CLASSIFIER ENSEMBLE

In [5]:
def classifier(secondary_dataset, config):
    
    # REQUESTED MODELS
    models = config['classification_ensemble']['models']
    
    # ENSEMBLE CONTAINER
    ensemble = [None] * len(models)

    # LOOP THROUGH MODELS
    for index, item in enumerate(config['classification_ensemble']['models']):

        # MODEL PARAMS
        name, settings = misc.key_value(item)
        
        # PRINT A MESSAGE
        print('TRAINING {} MODEL'.format(name.upper()))

        # TRAIN & PREDICT WITH THE MODEL
        model, predictions = training.start(secondary_dataset, name, settings)

        # APPEND RESULTS TO ENSEMBLE
        ensemble[index] = {
            'name': '{}_{}'.format(name, index),
            'predictions': predictions,
            'labels': secondary_dataset['test']['labels']
        }
    
    return ensemble